Competition: https://boosters.pro/championship/onetwotrip_challenge/overview
    

Solution was based on Alexander's Dyakonov (www.dyakonov.org/ag/) public benchmark - 

https://github.com/Dyakonov/notebooks/blob/master/dj_Benchmark_12trip.ipynb


In [2]:
import pandas as pd
import numpy as np
# для встроенных картинок
%pylab inline

import warnings
warnings.filterwarnings('ignore')
# progress-bar
from tqdm import tqdm, tqdm_notebook

Populating the interactive namespace from numpy and matplotlib


In [4]:
## Use pseudolabeling or not:
USE_PSEUDOLABELING = False

## if True, then use probabilities as pseudolabels
## if False, then use integer labels as pseudolabels
USE_SOFTLABELS = False

## Data loading

In [6]:
data_train = pd.read_csv('../data/onetwotrip_challenge_train.csv')
data_test = pd.read_csv('../data/onetwotrip_challenge_test.csv')
print ('Sizes:', data_train.shape, data_test.shape)

Sizes: (196056, 43) (455011, 37)


## Some statistics for data:

In [10]:
data_train.describe()

,orderid,field0,field1,field2,field3,field4,field5,field6,field7,field8,...,indicator_goal22,indicator_goal23,indicator_goal24,indicator_goal25,goal21,goal22,goal23,goal24,goal25,goal1
count,196056.000000,196056.000000,196056.000000,196056.000000,196056.000000,196056.000000,196056.000000,196056.000000,196056.000000,196056.000000,...,196056.000000,196056.000000,196056.000000,196056.000000,196056.000000,196056.000000,196056.000000,196056.000000,196056.000000,196056.000000
mean,98027.500000,28.413163,0.000912,7.980363,7.867864,3.528966,0.556214,1.577911,1.265292,0.879366,...,0.979649,0.348676,0.645754,0.647687,0.036627,0.168579,0.038713,0.022519,0.015098,0.022142
std,56596.636526,69.603944,1.009651,2.757008,3.013659,12.692057,0.496831,5.614900,0.441489,0.325702,...,0.141199,0.476552,0.478285,0.477692,0.187845,0.374381,0.192912,0.148365,0.121942,0.147144
min,0.000000,0.000000,-0.859222,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,49013.750000,0.000000,-0.471365,6.000000,6.000000,1.000000,0.000000,0.000000,1.000000,1.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,98027.500000,0.000000,-0.238651,8.000000,8.000000,1.000000,1.000000,0.000000,1.000000,1.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,147041.250000,13.000000,0.149205,10.000000,10.000000,2.000000,1.000000,0.000000,2.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,196055.000000,500.000000,82.142134,12.000000,12.000000,363.000000,1.000000,323.000000,2.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
data_test.describe()

,orderid,field0,field1,field2,field3,field4,field5,field6,field7,field8,...,field25,field26,field27,field28,field29,indicator_goal21,indicator_goal22,indicator_goal23,indicator_goal24,indicator_goal25
count,455011.000000,455011.000000,455011.000000,455011.000000,455011.000000,455011.000000,455011.000000,455011.000000,455011.000000,455011.000000,...,455011.000000,455011.000000,455011.000000,455011.000000,455011.000000,455011.000000,455011.000000,455011.000000,455011.000000,455011.000000
mean,227505.000000,28.415856,-0.000393,7.977884,7.867955,3.841116,0.556870,1.576944,1.262954,0.880027,...,4.859056,3.909336,1.754037,0.087103,2.979961,0.994994,0.979809,0.348244,0.645015,0.648055
std,131350.506005,69.747405,0.995814,2.753005,3.008647,19.976318,0.496756,5.555923,0.440238,0.324930,...,11.614136,4.264455,1.381171,0.368798,0.919229,0.070579,0.140652,0.476414,0.478509,0.477578
min,0.000000,0.000000,-0.859222,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,113752.500000,0.000000,-0.471365,6.000000,6.000000,1.000000,0.000000,0.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000
50%,227505.000000,0.000000,-0.238651,8.000000,8.000000,1.000000,1.000000,0.000000,1.000000,1.000000,...,1.000000,2.000000,1.000000,0.000000,3.000000,1.000000,1.000000,0.000000,1.000000,1.000000
75%,341257.500000,13.000000,0.149205,10.000000,10.000000,2.000000,1.000000,0.000000,2.000000,1.000000,...,1.000000,5.000000,3.000000,0.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,455010.000000,507.000000,79.271994,12.000000,12.000000,756.000000,1.000000,344.000000,2.000000,1.000000,...,193.000000,30.000000,7.000000,16.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### some simple FE (feature enginering):

In [14]:
full_data = pd.concat([data_train, data_test], axis=0, sort=False)

### We need to drop target values for 2nd task, otherwise it will entail overfitting

In [15]:
full_data.drop(['goal21'], inplace=True, axis=1)
full_data.drop(['goal22'], inplace=True, axis=1)
full_data.drop(['goal23'], inplace=True, axis=1)
full_data.drop(['goal24'], inplace=True, axis=1)
full_data.drop(['goal25'], inplace=True, axis=1)

0, 1, 2, 3, 4, 12, 15, 18, 21, 29 

In [17]:
%%time

group_by_features = ['userid', 'field0', 'field1',
                     'field2', 'field3', 'field4', 'field6', 'field9', 'field11',
                     'field12', 'field15', 'field18',
                     'field21', 'field29', 'indicator_goal21', 'indicator_goal22', 'indicator_goal23',
                     'indicator_goal24', 'indicator_goal25']
cols = full_data.columns.delete(1)[:-1]

for group_by_feature in group_by_features:
    for col in tqdm_notebook(cols):
        mean_per_col = full_data.groupby([group_by_feature])[
            col].transform('mean').replace(-np.inf, np.nan).replace(np.inf, np.nan).astype(np.float32)
        std_per_col = full_data.groupby([group_by_feature])[
            col].transform('std').replace(-np.inf, np.nan).replace(np.inf, np.nan).astype(np.float32)
        full_data[f'{col}_to_mean_{group_by_feature}'] = full_data[col] / mean_per_col
        full_data[f'{col}_to_std_{group_by_feature}'] = full_data[col] / std_per_col
        


CPU times: user 4min 27s, sys: 2min 32s, total: 7min
Wall time: 6min 56s


Features, which named as `indicator_goal2i`, $i \in {1, 2, 3, 4, 5}$, looks like iteractions of user with items, so I've  decided to use matrix factorization to get user embeddings and use it as extra features.

### Data preparation:

In [18]:
full_data['indicator_goal_label_encoded'] = np.where(full_data['indicator_goal21'], 0, 5)
full_data['indicator_goal_label_encoded'] = np.where(full_data['indicator_goal22'], 1,
                                                     full_data['indicator_goal_label_encoded'])
full_data['indicator_goal_label_encoded'] = np.where(full_data['indicator_goal23'], 2,
                                                     full_data['indicator_goal_label_encoded'])
full_data['indicator_goal_label_encoded'] = np.where(full_data['indicator_goal24'], 3,
                                                     full_data['indicator_goal_label_encoded'])
full_data['indicator_goal_label_encoded'] = np.where(full_data['indicator_goal25'], 4,
                                                     full_data['indicator_goal_label_encoded'])

In [19]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

users = le.fit_transform(full_data['userid'].values.reshape(-1))
items = full_data['indicator_goal_label_encoded'].values.reshape(-1)
values = np.ones_like(users)

In [20]:
from scipy.sparse import coo_matrix

user_item = coo_matrix((values, (users, items)))

###  ALS matrix factorization:

In [22]:
import implicit

n_factors = 150
model = implicit.als.AlternatingLeastSquares(factors=n_factors, iterations=120)
model.fit(user_item.T.tocsr())

GPU training requires factor size to be a multiple of 32. Increasing factors from 150 to 160.


In [23]:
%%time

le.inverse_transform(users)

CPU times: user 71.1 ms, sys: 0 ns, total: 71.1 ms
Wall time: 70.5 ms


array(['10d654494cbe97bbb25d51ead2600679aff9e097924add09d8066010a0c9adaf',
       '4aafc0391f72bbcf60537aece62923baf9ce644b64ac365a75e021cca794b91b',
       'bac8ffef46348f587c8d17137ab01fb24aef21547c647d2ea68a0214c8abf003',
       ...,
       '255bfc6ac2f9c98d7f86e59bd72a682e7a50c2db6107495fabd8729471cc172e',
       '58a0463aea3ece956b49099380c17cf1d3dd591b70e58c080600920c51af9b9a',
       '7db5e20ad97cbec3604f8cff2ebc407b4f3f1a29c45631c52d85ac6a13501fa0'],
      dtype=object)

In [24]:
for factor in tqdm_notebook(range(n_factors)):
    full_data['factor_als_'+str(factor)] = model.user_factors[users, factor]
    full_data['factor_als_'+str(factor)] = model.user_factors[users, factor]

## BPR matrix factorization:

In [25]:
%time

model = implicit.bpr.BayesianPersonalizedRanking(factors=n_factors, iterations=120, use_gpu=False)
model.fit(user_item.T.tocsr())

for factor in tqdm_notebook(range(n_factors)):
    full_data['factor_bpr_'+str(factor)] = model.user_factors[users, factor]
    full_data['factor_bpr_'+str(factor)] = model.user_factors[users, factor]

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 4.05 µs


In [26]:
full_data.shape

(651067, 1707)

In [41]:
data_train = full_data[~full_data['goal1'].isna()]
data_test = full_data[full_data['goal1'].isna()]

## Drop target from training set

In [42]:
y = data_train.pop('goal1')
y = y.values

In [44]:
data_train.head()

,orderid,userid,field0,field1,field2,field3,field4,field5,field6,field7,...,factor_bpr140,factor_bpr141,factor_bpr142,factor_bpr143,factor_bpr144,factor_bpr145,factor_bpr146,factor_bpr147,factor_bpr148,factor_bpr149
0,0,10d654494cbe97bbb25d51ead2600679aff9e097924add...,0,-0.626508,11,12,1,1,0,1,...,0.007633,0.015211,-0.024880,0.049655,0.000328,-0.028086,-0.029308,-0.010251,-0.026921,0.005300
1,1,4aafc0391f72bbcf60537aece62923baf9ce644b64ac36...,144,-0.393794,5,7,2,0,0,2,...,-0.016551,-0.016767,0.076734,-0.157866,0.030844,0.037039,0.074635,0.009446,0.034014,-0.030779
2,2,bac8ffef46348f587c8d17137ab01fb24aef21547c647d...,134,-0.548937,2,3,2,0,0,1,...,0.010243,0.008635,-0.027070,0.046813,0.003109,-0.025548,-0.027793,-0.010233,-0.026142,0.007182
3,3,0392247b4b87674aba2c32bf2292b105771a6a376871be...,0,-0.238651,10,11,1,1,3,2,...,0.005618,0.012722,-0.023667,0.049049,-0.000563,-0.023540,-0.031876,-0.012161,-0.025659,0.003450
4,4,d1aeefef311bbeb4bd84876c8d49421f276674527d5578...,0,-0.704079,8,11,1,1,0,1,...,0.009037,0.012994,-0.022391,0.043133,0.002050,-0.021355,-0.027301,-0.007912,-0.025380,0.006430


In [47]:
train_userid = data_train['userid']
data_train.drop(columns=['orderid'], inplace=True)
ids = data_test.pop('orderid')

In [48]:
data_test.drop(['userid'], inplace=True, axis=1)
data_test.drop(['goal1'], inplace=True, axis=1)

In [49]:
cols = data_test.columns
cols

Index(['field0', 'field1', 'field2', 'field3', 'field4', 'field5', 'field6',
       'field7', 'field8', 'field9',
       ...
       'factor_bpr140', 'factor_bpr141', 'factor_bpr142', 'factor_bpr143',
       'factor_bpr144', 'factor_bpr145', 'factor_bpr146', 'factor_bpr147',
       'factor_bpr148', 'factor_bpr149'],
      dtype='object', length=1704)

In [50]:
data_train = data_train[cols]

## Experiments:

Use super simple validation:

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_train, y, test_size=0.3, random_state=13)

In [52]:
from sklearn.model_selection import GroupKFold, StratifiedKFold

N_SPLITS = 5
gr_kfold = StratifiedKFold(n_splits=N_SPLITS)

In [53]:
import lightgbm as lgb
model = lgb.LGBMClassifier(bagging_fraction=0.5539674763124854, bagging_freq=5,
               boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.7841488307689868, eval_metric='AUC',
               importance_type='split', lambda_l1=7.892207361708597,
               lambda_l2=8.705620624758234, learning_rate=0.03909410437165517,
               max_bin=270, max_depth=10, metric='AUC', min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
               n_jobs=16, num_leaves=15, objective='binary', random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, seed=42, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0,
               verbose=0)

In [54]:
%%time

param = {'bagging_fraction': 0.5539674763124854, 'bagging_freq':5,
               'boosting_type': 'gbdt', 'class_weight':None,
               'colsample_bytree': 0.7841488307689868, 'eval_metric': 'AUC',
               'importance_type': 'split', 'lambda_l1': 7.892207361708597,
               'lambda_l2': 8.705620624758234, 'learning_rate': 0.03909410437165517,
               'max_bin': 270, 'max_depth': 10, 'metric': 'AUC', 'min_child_samples': 20,
               'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 2000,
               'n_jobs': 16, 'num_leaves': 15, 'objective': 'binary', 'random_state': 13,
               'seed': 13, 'silent': False,
               'verbose': 100, 'early_stopping_rounds': 200}
param['metric'] = 'auc'
oof_predictions = np.zeros(data_train.shape[0])

for train_idxs, test_idxs in gr_kfold.split(data_train, y):
    q = lgb.train(param, train_set=lgb.Dataset(data_train.values[train_idxs], y[train_idxs]), num_boost_round=500,
                  valid_sets=[lgb.Dataset(data_train.values[train_idxs], y[train_idxs]),
                              lgb.Dataset(data_train.values[test_idxs], y[test_idxs])],
                 verbose_eval=500)
    oof_predictions[test_idxs] = q.predict(data_train.values[test_idxs])

Training until validation scores don't improve for 200 rounds
[500]	valid_0's auc: 0.86873	valid_1's auc: 0.692603
Early stopping, best iteration is:
[604]	valid_0's auc: 0.887849	valid_1's auc: 0.694016
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[154]	valid_0's auc: 0.774026	valid_1's auc: 0.71287
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[247]	valid_0's auc: 0.805105	valid_1's auc: 0.714469
Training until validation scores don't improve for 200 rounds
[500]	valid_0's auc: 0.869568	valid_1's auc: 0.700762
Early stopping, best iteration is:
[469]	valid_0's auc: 0.862441	valid_1's auc: 0.701812
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[189]	valid_0's auc: 0.786063	valid_1's auc: 0.703601
CPU times: user 1h 13min 18s, sys: 15.1 s, total: 1h 13min 33s
Wall time: 5min 17s


In [55]:
from sklearn.metrics import roc_auc_score

In [56]:
roc_auc_score(y, oof_predictions[:data_train.shape[0]])

0.7042638537057597

0.7002432786953284

Use bayes_opt library for tuning model parameters:

In [57]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score


def lgb_bayesian(max_depth,
                 lambda_l1,
                 lambda_l2,
                 bagging_fraction,
                 bagging_freq,
                 colsample_bytree,
                 learning_rate,
                 num_leaves,
                 max_bin
                      ):
    params = {
        'boosting_type': 'gbdt',
        'metric': 'AUC',
        'objective': 'binary',
        'eval_metric': 'AUC',
        'n_jobs': 16,
        'seed': 42,
        'early_stopping_rounds': 150,
        'n_estimators': 2000,
        'learning_rate': learning_rate,
        'max_depth': int(max_depth),
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': int(bagging_freq),
        'colsample_bytree': colsample_bytree,
        'verbose': 0,
        'early_stopping_rounds': 200
    }
    oof_predictions = np.zeros(data_train.shape[0])
    for train_idxs, test_idxs in gr_kfold.split(data_train, y):
        model = lgb.LGBMClassifier(boosting_type=params['boosting_type'],
                                    metric='AUC',
                                    objective='binary',
                                    eval_metric='AUC',
                                    n_jobs= 16,
                                    seed=13,
                                    n_estimators= 2000,
                                    learning_rate=learning_rate,
                                    max_depth=int(max_depth),
                                    lambda_l1=lambda_l1,
                                    lambda_l2=lambda_l2,
                                    bagging_fraction=bagging_fraction,
                                    bagging_freq=int(bagging_freq),
                                    max_bin=int(max_bin),
                                    num_leaves=int(num_leaves),
                                    colsample_bytree=colsample_bytree,
                                    verbose=0
                                    )
        model.fit(data_train.values[train_idxs],
                  y[train_idxs],
                  early_stopping_rounds=200,
                  eval_metric='AUC',
                  eval_set=(data_train.values[test_idxs], y[test_idxs]), verbose=0)
        oof_predictions[test_idxs] = model.predict_proba(data_train.values[test_idxs])[:, 1]
    return roc_auc_score(y, oof_predictions)

In [58]:
bounds_LGB = {
    'max_depth': (2, 25),
    'lambda_l1': (0, 10),
    'lambda_l2': (0, 10),
    'bagging_fraction': (0.4, 0.9),
    'bagging_freq': (1, 10),
    'colsample_bytree': (0.4, 0.9),
    'learning_rate': (0.025, 0.1),
    'num_leaves': (2, 20),
    'max_bin': (2, 1000)
}


lgbm_BO = BayesianOptimization(lgb_bayesian, bounds_LGB, random_state=13 * 2)

In [ ]:
%%time

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    lgbm_BO.maximize(init_points=10, n_iter=10, xi=0, alpha=1e-3)

|   iter    |  target   | baggin... | baggin... | colsam... | lambda_l1 | lambda_l2 | learni... |  max_bin  | max_depth | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7037   |  0.554    |  5.675    |  0.7841   |  7.892    |  8.706    |  0.03909  |  271.0    |  13.41    |  15.3     |
|  2        |  0.7043   |  0.4975   |  2.618    |  0.6694   |  2.828    |  7.609    |  0.04523  |  294.9    |  24.14    |  18.44    |
|  3        |  0.7054   |  0.6841   |  5.369    |  0.7269   |  5.366    |  9.957    |  0.03565  |  975.4    |  16.25    |  14.97    |
|  4        |  0.7015   |  0.5475   |  6.401    |  0.7753   |  6.414    |  9.233    |  0.09165  |  645.4    |  8.8      |  14.29    |
|  5        |  0.7003   |  0.5385   |  2.138    |  0.685    |  4.091    |  5.604    |  0.08908  |  98.53    |  10.15    |  19.53    |
|  6        |  0.7067   |  0.861    |  9.718    |  0.4255   | 

In [ ]:
model_params = {
    'max_depth': int(lgbm_BO.max['params']['max_depth']),
    'learning_rate': lgbm_BO.max['params']['learning_rate'],
    'num_leaves': int(lgbm_BO.max['params']['num_leaves']),
    'max_bin': int(lgbm_BO.max['params']['max_bin']),
    'boosting_type': 'gbdt',
    'metric': 'AUC',
    'objective': 'binary',
    'eval_metric': 'AUC',
    'n_jobs': 16,
    'seed': 42,
    'n_estimators': 2000,
    'lambda_l1': lambda_l1,
    'lambda_l2': lambda_l2,
    'bagging_fraction': lgbm_BO.max['params']['bagging_fraction'],
    'bagging_freq': int(lgbm_BO.max['params']['bagging_freq']),
    'colsample_bytree': lgbm_BO.max['params']['colsampel_bytree'],
    'verbose': 0
}

In [ ]:
import sklearn.utils

if USE_PSEUDOLABELING:
    pseudolabels = pd.read_csv('./pseudolabels_692673.csv', index_col=0)
    data_train = pd.concat([data_train, data_test[(pseudolabels['proba'].values.reshape(-1) > 0.99) | (pseudolabels['proba'].values.reshape(-1) < 0.01)]], sort=False).reset_index(drop=True)
    pseudolabels = pseudolabels[(pseudolabels['proba'].values.reshape(-1) > 0.99) | (pseudolabels['proba'].values.reshape(-1) < 0.01)]
    y = np.hstack([y.reshape(-1), pseudolabels['proba'].values])
    if not USE_SOFTLABELS:
        y = np.where(y > 0.8, 1, 0)
    data_train, y = sklearn.utils.shuffle(data_train, y, random_state=13)

In [ ]:
if USE_PSEUDOLABELING:
    data_test[(pseudolabels['proba'].values.reshape(-1) > 0.99) \
              | (pseudolabels['proba'].values.reshape(-1) < 0.01)].shape

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
oof_predictions = np.zeros(data_train.shape[0])
test_predictions = np.zeros(data_test.shape[0])
for train_idxs, test_idxs in tqdm_notebook(gr_kfold.split(data_train, y)):
    model = lgb.LGBMClassifier(
        max_depth=int(lgbm_BO.max['params']['max_depth']),
        learning_rate=lgbm_BO.max['params']['learning_rate'],
        num_leaves=int(lgbm_BO.max['params']['num_leaves']),
        max_bin=int(lgbm_BO.max['params']['max_bin']),
        boosting_type='gbdt',
        metric='AUC',
        objective='binary',
        eval_metric='AUC',
        n_jobs=16,
        seed=42,
        n_estimators=2000,
        lambda_l1=lgbm_BO.max['params']['lambda_l1'],
        lambda_l2=lgbm_BO.max['params']['lambda_l2'],
        bagging_fraction=lgbm_BO.max['params']['bagging_fraction'],
        bagging_freq=int(lgbm_BO.max['params']['bagging_freq']),
        colsample_bytree=lgbm_BO.max['params']['colsample_bytree'],
        verbose=0
    )
    model.fit(data_train.values[train_idxs],
              y[train_idxs],
              early_stopping_rounds=200,
              eval_metric='AUC',
              eval_set=(data_train.values[test_idxs], y[test_idxs]), verbose=0)
    oof_predictions[test_idxs] = model.predict_proba(data_train.values[test_idxs])[:, 1]
    test_predictions += model.predict_proba(data_test.values)[:, 1] / N_SPLITS
print('OOF: \n' + str(roc_auc_score(y, oof_predictions)))


OOF: 
0.7655344173061015 - pseudolabeling

OOF: 
0.7037542343142662

In [ ]:
pd.DataFrame(test_predictions,  columns=['proba'], index=ids.values).to_csv('lgbm_1500features_als_bpr.csv')

## Try to use xgboost:

In [ ]:
%%time

from xgboost import XGBClassifier

In [50]:
oof_predictions = np.zeros(data_train.shape[0])
test_predictions = np.zeros(data_test.shape[0])
for train_idxs, test_idxs in tqdm_notebook(gr_kfold.split(data_train, y)):
    model = XGBClassifier(gpu_hist=True, nthread=16, booster='dart', objective='binary:logistic')
    model.fit(data_train.values[train_idxs],
              y[train_idxs], 
             )
    oof_predictions[test_idxs] = model.predict_proba(data_train.values[test_idxs])[:, 1]
    test_predictions += model.predict_proba(data_test.values)[:, 1] / N_SPLITS
print('OOF: \n' + str(roc_auc_score(y, oof_predictions)))


OOF: 
0.6977229857897738


OOF: 
0.6969501317966372

In [51]:
pd.DataFrame(test_predictions,  columns=['proba'], index=ids.values).to_csv('xgb_1500features_als_bpr.csv')

## Some experiments with catboost

In [ ]:
from catboost import CatBoostClassifier


def catboost_bayesian(max_depth,
                      n_estimators,
                      lr,
                      ):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    model = CatBoostClassifier(max_depth=max_depth, n_estimators=n_estimators,
                               learning_rate=lr, objective='CrossEntropy', eval_metric='AUC',
                               task_type='GPU', verbose=0, thread_count=16)
    model.fit(X_train, y_train)

    return roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

In [ ]:
bounds_cat = {
    'max_depth': (1, 16),
    'n_estimators': (1, 1000),
    'lr': (1e-3, 1),
}

cat_BO = BayesianOptimization(catboost_bayesian, bounds_cat, random_state=13 * 2)

In [ ]:
%%time
import warnings


with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    cat_BO.maximize(init_points=20, n_iter=20, acq='ucb')

In [ ]:
model = CatBoostClassifier(max_depth=int(cat_BO.max['params']['max_depth']),
                           n_estimators=int(cat_BO.max['params']['n_estimators']),
                           learning_rate=cat_BO.max['params']['lr'],
                           objective='CrossEntropy', eval_metric='AUC',
                               task_type='GPU', verbose=0, thread_count=16)

In [ ]:
model.fit(data_train, y)

In [ ]:
a = model.predict_proba(data_test)[:, 1] # вероятности за 1й класс

In [ ]:
pd.DataFrame(a,  columns=['proba'], index=ids.values).to_csv('ctb.csv')

## Fitting and predicting:

In [ ]:
model.fit(data_train, y)

In [ ]:
a = model.predict_proba(data_test)[:, 1] # вероятности за 1й класс

In [ ]:
pd.DataFrame(a,  columns=['proba'], index=ids.values).to_csv('dj1_01_.csv')